# Gentle introduction to Ray datasets APIs

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives

In this introductory tutorial you will learn to:
 * create, transform, read, and save Ray datasets
 * use shards for parallel processing of large datasets
 * understand datapipelines and their merits
 * use `DatasetPipeline` for parallel computation 
 * use datasets for last-mile ML ingestion for distributed training
 * why use datasets and what for

### Overview

This is a brief and gentle introduction to Ray's native library `ray dataset`. As a native Ray library, built atop Ray, it allows you to exchange data among Ray tasks, actors, libraries, and applications. It also allows you to read/write training data from different file sources, include csv, parquet, text, etc.

Ray Datasets, using distributed [Apache Arrow](https://arrow.apache.org/), are designed to load and preprocess data for distributed ML training pipelines. Compared to other loading solutions, Datasets are more flexible (e.g., you can express higher-quality per-epoch global shuffles) and provides higher overall performance.

Additionally, Ray datasets provides standard and simple transformations like `map`, `filter`, and `partition`. Ray datasets is *not* a replacement for a full-fledged data processing library for doing exploratory data analysis (EDA), extract, transform and load (ETL) or a subsitute for Apache Spark or Dask or Pandas DataFrames. Its primary objective is the last-mile rudimentary distributed data preprocessing and data ingestion for distributed ML training by providing simple primittive transformational functions.

Supporting myriad [file formats and data sources](https://docs.ray.io/en/latest/data/dataset.html#datasource-compatibility), you can read from and write to local FS and cloud storage. 

<img src="images/dataset.png" width="70%" height="35%">


### Key concepts

To work with Ray Datasets, you need to understand how `Datasets` and `Dataset Pipelines` work. That is, how datasets are stored internally and in what format. And what benefit does `Datapipelines` offer for faster processing and execution. A quick peek into each of these will shed some light into overall benefits of Ray Datasets.

Let's start with the internal format. 

#### Ray Datasets

A Ray dataset implements a distributed [Apache Arrow](https://arrow.apache.org/). As such, a Dataset consists of a list of Ray object references to blocks. Each block holds a set of items in either an [Arrow table](https://arrow.apache.org/docs/python/data.html#tables) (when created from or transformed to tabular or tensor data), a Pandas DataFrame (when created from or transformed to Pandas data), or a Python list (otherwise).

<img src="images/dataset-arch.png" width="70%" height="35%">

#### Dataset Pipelines
Datasets execute their transformations synchronously or eagerly in blocking calls. However, it can be useful to overlap dataset computations with output. This can be done with a `DatasetPipeline`.

A `DatasetPipeline` is a unified iterator over a (potentially infinite) sequence of Ray Datasets, each of which represents a window over the original data. Conceptually, it is similar to a `Spark DStream`, but manages execution over a bounded amount of source data instead of an unbounded stream. Ray computes each dataset window on-demand and stitches their output together into a single logical data iterator. `DatasetPipeline` implements most of the same transformation and output methods as Datasets (e.g., `map`, `filter`, `split`, `iter_rows`, `to_torch`, etc.).

<img src="images/ray_dataset_pipeline.jpg" width="75%" height="45%">

### Datasets Execution Model
This section overviews the execution model of Datasets, which may be useful for understanding and tuning performance.

#### Reading Data
Datasets uses Ray tasks, for parallelism, to read data from remote storage or source. When reading from a file-based datasource (e.g., S3, GCS), it creates a number of parallel
read tasks equal to the specified read parallelism (200 by default). One or more files will be assigned to each read task. Each read task reads its assigned files and produces one or more output blocks (Ray objects):

<img src="https://docs.ray.io/en/master/_images/dataset-read.svg" height="25%" width="50%">

In the common case, each read task produces a single output block. Read tasks may split the output into multiple blocks if the data exceeds the target max block size (2GiB by default). This automatic block splitting avoids out-of-memory errors when reading very large single files (e.g., a 100-gigabyte CSV file). All of the built-in datasources except for JSON currently support automatic block splitting.

#### Deferred Read Task Execution

When a Dataset is created using `ray.data.read_*`, only the first read task will be executed initially. This avoids blocking Dataset creation on the reading of all data files, enabling inspection functions like `ds.schema()` without incurring high read costs. `<ray.data.Dataset.schema>`() and `ds.show()` can be used right away. Executing further transformations on the Dataset will trigger execution of all read tasks.

#### Dataset Transforms

Datasets use either Ray tasks or Ray actors to transform datasets (i.e., for `ds.map_batches()`, `ds.map()`, or `ds.flat_map()`). By default, tasks are used `(compute="tasks")`. Actors can be specified with `compute="actors"`, in which case an autoscaling pool of Ray actors will be used to apply transformations. Using actors allows for expensive state initialization (e.g., for GPU-based tasks) to be re-used. Whichever compute strategy is used, each map task generally takes in one block and produces one or more output blocks. The output block splitting rule is the same as for file reads (blocks are split after hitting the target max block size of 2GiB):

<img src="https://docs.ray.io/en/master/_images/dataset-map.svg" height="25%" width="50%">

#### Shuffling Data

Certain operations like `ds.sort()` and `ds.groupby()` require data blocks to be partitioned by value. Datasets executes this in three phases. 

First, a wave of sampling tasks determines suitable partition boundaries based on a random sample of data. Second, map tasks divide each input block into a number of output blocks equal to the number of reduce tasks. Third, reduce tasks take assigned output blocks from each map task and combines them into one block. Overall, this strategy generates O(n^2) intermediate objects where n is the number of input blocks.

You can also change the partitioning of a Dataset using `ds.random_shuffle()` or `ds.repartition()`. The former should be used if you want to randomize the order of elements in the dataset. The second should be used if you only want to equalize the size of the Dataset blocks (e.g., after a read or transformation that may skew the distribution of block sizes). Note that repartition has two modes, `shuffle=False`, which performs the minimal data movement needed to equalize block sizes, and `shuffle=True`, which performs a full (non-random) distributed shuffle:

<img src="https://docs.ray.io/en/master/_images/dataset-shuffle.svg" height="25%" width="50%">

#### Fault tolerance

Datasets relies on task-based 🤹‍♀️ [fault tolerance](https://docs.ray.io/en/latest/ray-core/tasks/fault-tolerance.html) in Ray core. Specifically, a `Dataset` will be automatically recovered by Ray in case of failures. This works through **lineage reconstruction**: a Dataset is a collection of Ray objects stored in shared memory, and if any of these objects are lost, then Ray will recreate them by re-executing the task(s) that created them.

There are a few cases that are not currently supported: 

 1. If the original creator of the Dataset dies ☠️. This is because the creator stores the metadata for the objects that comprise the Dataset. 
 2. For a `DatasetPipeline.split()` 👯‍♂️, we do not support recovery for a consumer failure. When there are multiple consumers, they must all read the split pipeline in lockstep. To recover from this case, the pipeline and all consumers must be restarted together. 
 3. The `compute=actors`🧑‍🏭 option for transformations.

#### Execution and Memory Management

See [Execution and Memory Management](https://docs.ray.io/en/master/data/memory-management.html#data-advanced) for more details about how Datasets manages memory and optimizations such as lazy vs eager execution.

In [1]:
import logging, os, random, warnings
import ray

In [2]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
if ray.is_initialized:
    ray.shutdown()
ctx = ray.init(logging_level=logging.ERROR)
print(ctx)

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-15_16-31-19_707528_48020/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-15_16-31-19_707528_48020/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-07-15_16-31-19_707528_48020', 'metrics_export_port': 52224, 'gcs_address': '127.0.0.1:58863', 'address': '127.0.0.1:58863', 'node_id': '3dd043338e210bfae9b32d7b1140c3253014acdbee6b00ec06e0cfd9'})


In [4]:
print(f"Dashboard url: http://{ctx.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8265


### Creating a simple Ray Dataset

For quick illustration, let's create a generic dataset of 1K integers and look at the schema and underlying datatype. 

In [5]:
ds = ray.data.range(1000)
ds.count()

1000

In [6]:
ds.schema()

int

The difference between `show` and `take` is that the former takes one item at time and prints it, while the latter iterates over row items from the dataset, appends to a list and returns it. Underneath, `ds.show()` calls `ds.take()`.

In [7]:
ds.show(5)

0
1
2
3
4


In [8]:
ds.take(5)

[0, 1, 2, 3, 4]

### Creating a large Ray Dataset

Let's create a synthetic dataset, *Homeowners*, of Arrow records (800K) with several columns and data associated with it. 

To illustrate some simple transformational functions, we'll use this generated data.

In [10]:
NUM_ROWS = 800_001
STATES = ["CA", "AZ", "OR", "WA", "TX", "UT", "NV", "NM"]
M_STATUS = ["married", "single", "domestic", "divorced", "undeclared"]
GENDER = ["F", "M", "U"]
HOME_OWNER = ["condo", "house", "rental", "cottage"]

items = [{"id": i,
          "ssn": None,
          "name": None,
          "amount": i * 1.5, 
          "interest": random.randint(1,5) * .1,
          "state": random.choice(STATES),
          "marital_status": random.choice(M_STATUS),
          "property": random.choice(HOME_OWNER),
          "dependents": random.randint(1, 5),
          "defaulted": random.randint(0,1),
          "gender":random.choice(GENDER) } for i in range(1,NUM_ROWS)]
items[:2]

[{'id': 1,
  'ssn': None,
  'name': None,
  'amount': 1.5,
  'interest': 0.2,
  'state': 'AZ',
  'marital_status': 'single',
  'property': 'rental',
  'dependents': 3,
  'defaulted': 1,
  'gender': 'M'},
 {'id': 2,
  'ssn': None,
  'name': None,
  'amount': 3.0,
  'interest': 0.1,
  'state': 'CA',
  'marital_status': 'married',
  'property': 'house',
  'dependents': 4,
  'defaulted': 1,
  'gender': 'F'}]

#### Creating a dataset from list of dictionary items

Ray data can be created of a dictionary of items. 

*Questions: how is num_blocks computed?*

In [15]:
arrow_ds = ray.data.from_items(items)
arrow_ds

Dataset(num_blocks=200, num_rows=800000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})

In [16]:
arrow_ds.count()

800000

In [17]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': None,
           'name': None,
           'amount': 1.5,
           'interest': 0.2,
           'state': 'AZ',
           'marital_status': 'single',
           'property': 'rental',
           'dependents': 3,
           'defaulted': 1,
           'gender': 'M'})]

In [18]:
arrow_ds.schema()

id: int64
ssn: null
name: null
amount: double
interest: double
state: string
marital_status: string
property: string
dependents: int64
defaulted: int64
gender: string

### Saving datasets and reading as a parquet files 🗃
Ray datasets support myriad data formats. Let's save this dataset as a parquet file and create `N` partitions.

In [21]:
path = os.path.abspath("data_homeowners/interest.parquet")
arrow_ds.repartition(5).write_parquet(path)

Write Progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 95.87it/s]


In [22]:
!ls -l data_homeowners/interest.parquet

total 21624
-rw-r--r--  1 jules  staff  2207898 Jul 15 16:48 cc5dfc165fcc458fa25183b050fc9ddb_000000.parquet
-rw-r--r--  1 jules  staff  2189245 Jul 15 16:48 cc5dfc165fcc458fa25183b050fc9ddb_000001.parquet
-rw-r--r--  1 jules  staff  2187507 Jul 15 16:48 cc5dfc165fcc458fa25183b050fc9ddb_000002.parquet
-rw-r--r--  1 jules  staff  2186992 Jul 15 16:48 cc5dfc165fcc458fa25183b050fc9ddb_000003.parquet
-rw-r--r--  1 jules  staff  2287137 Jul 15 16:48 cc5dfc165fcc458fa25183b050fc9ddb_000004.parquet


In [23]:
arrow_ds = ray.data.read_parquet(path)

In [25]:
arrow_ds.take(1)

[ArrowRow({'id': 1,
           'ssn': None,
           'name': None,
           'amount': 1.5,
           'interest': 0.2,
           'state': 'AZ',
           'marital_status': 'single',
           'property': 'rental',
           'dependents': 3,
           'defaulted': 1,
           'gender': 'M'})]

### Transforming data with simple methods

Ray datasets support transformation in parallel using `map`. It uses Ray tasks to execute eagerly or synchronously. Among others [transformations](https://docs.ray.io/en/latest/data/package-ref.html#dataset-api), it supports`filter`, `flat_map`, `groupBy`etc.

Let's try a using `.map()`, `.filter()` and `.groupBy` on our dataset. 

The `map()` and `filter()` are row-based operations. This can be expensive for large datasets. However, you can use `map_batches(...)` with batch_size=4096 as default. This will create a task per block and each batch will be vectorized and executed in parallel. Ray tasks are created per block for a map operation. 

Let's try first with row-based transformation

In [26]:
%%time
arrow_ds.filter(lambda x: x['amount'] > 10000).take(3)

Read->Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.13it/s]

CPU times: user 51.3 ms, sys: 21.8 ms, total: 73.1 ms
Wall time: 4.44 s


[ArrowRow({'id': 6667,
           'ssn': None,
           'name': None,
           'amount': 10000.5,
           'interest': 0.2,
           'state': 'NV',
           'marital_status': 'single',
           'property': 'house',
           'dependents': 1,
           'defaulted': 0,
           'gender': 'U'}),
 ArrowRow({'id': 6668,
           'ssn': None,
           'name': None,
           'amount': 10002.0,
           'interest': 0.4,
           'state': 'NM',
           'marital_status': 'single',
           'property': 'cottage',
           'dependents': 4,
           'defaulted': 0,
           'gender': 'U'}),
 ArrowRow({'id': 6669,
           'ssn': None,
           'name': None,
           'amount': 10003.5,
           'interest': 0.4,
           'state': 'CA',
           'marital_status': 'undeclared',
           'property': 'rental',
           'dependents': 5,
           'defaulted': 1,
           'gender': 'U'})]

Let's try a `.map_batches()`, which is vectorized. We should expect faster execution. 

*Question: Why the `.map()` returned `ArrowRow` and `.map_batches` returned `PandasRow`*?

In [27]:
%%time
arrow_ds.map_batches(lambda df: df[df["amount"] > 10000]).take(3)

Read->Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 17.79it/s]

CPU times: user 41 ms, sys: 13.2 ms, total: 54.2 ms
Wall time: 315 ms


[PandasRow({'id': 6667,
            'ssn': None,
            'name': None,
            'amount': 10000.5,
            'interest': 0.2,
            'state': 'NV',
            'marital_status': 'single',
            'property': 'house',
            'dependents': 1,
            'defaulted': 0,
            'gender': 'U'}),
 PandasRow({'id': 6668,
            'ssn': None,
            'name': None,
            'amount': 10002.0,
            'interest': 0.4,
            'state': 'NM',
            'marital_status': 'single',
            'property': 'cottage',
            'dependents': 4,
            'defaulted': 0,
            'gender': 'U'}),
 PandasRow({'id': 6669,
            'ssn': None,
            'name': None,
            'amount': 10003.5,
            'interest': 0.4,
            'state': 'CA',
            'marital_status': 'undeclared',
            'property': 'rental',
            'dependents': 5,
            'defaulted': 1,
            'gender': 'U'})]

You can see that `.map_batches()` is a lot faster than row based. So for large datasets use 
`.map_batches()`.

Let's try a filter operation: both per row operation and per block as vectorized

In [28]:
%%time
arrow_ds.filter(lambda x: x['amount'] > 10000.00 and x['state'] == 'CA').take(2)

Read->Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.89it/s]

CPU times: user 22.4 ms, sys: 9.65 ms, total: 32 ms
Wall time: 1.74 s


[ArrowRow({'id': 6669,
           'ssn': None,
           'name': None,
           'amount': 10003.5,
           'interest': 0.4,
           'state': 'CA',
           'marital_status': 'undeclared',
           'property': 'rental',
           'dependents': 5,
           'defaulted': 1,
           'gender': 'U'}),
 ArrowRow({'id': 6675,
           'ssn': None,
           'name': None,
           'amount': 10012.5,
           'interest': 0.30000000000000004,
           'state': 'CA',
           'marital_status': 'undeclared',
           'property': 'house',
           'dependents': 5,
           'defaulted': 1,
           'gender': 'F'})]

In [29]:
%%time
arrow_ds.map_batches(lambda df: df[[df["amount"] > 10000] and df["state"] == "CA"]).take(3)

Read->Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 42.48it/s]

CPU times: user 27.6 ms, sys: 10 ms, total: 37.6 ms
Wall time: 138 ms


[PandasRow({'id': 2,
            'ssn': None,
            'name': None,
            'amount': 3.0,
            'interest': 0.1,
            'state': 'CA',
            'marital_status': 'married',
            'property': 'house',
            'dependents': 4,
            'defaulted': 1,
            'gender': 'F'}),
 PandasRow({'id': 4,
            'ssn': None,
            'name': None,
            'amount': 6.0,
            'interest': 0.1,
            'state': 'CA',
            'marital_status': 'divorced',
            'property': 'house',
            'dependents': 2,
            'defaulted': 0,
            'gender': 'F'}),
 PandasRow({'id': 9,
            'ssn': None,
            'name': None,
            'amount': 13.5,
            'interest': 0.1,
            'state': 'CA',
            'marital_status': 'domestic',
            'property': 'rental',
            'dependents': 1,
            'defaulted': 1,
            'gender': 'U'})]

Use `groupBy` state and compute the count

Under the hood is distributed parallel group and vectorized.

In [30]:
results = arrow_ds.groupby("state").count()

Shuffle Reduce: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1186.51it/s]


In [31]:
results.show()

{'state': 'AZ', 'count()': 99980}
{'state': 'CA', 'count()': 99766}
{'state': 'NM', 'count()': 99959}
{'state': 'NV', 'count()': 99710}
{'state': 'OR', 'count()': 99684}
{'state': 'TX', 'count()': 100184}
{'state': 'UT', 'count()': 100402}
{'state': 'WA', 'count()': 100315}


Get the max of certain columns

In [32]:
results = arrow_ds.max(["amount", "interest", "dependents"])
results

Shuffle Reduce: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 330.26it/s]


ArrowRow({'max(amount)': 1200000.0,
          'max(interest)': 0.5,
          'max(dependents)': 5})

### Accessing datasets using batches or iterating by rows

Datasets can be passed to Ray tasks or actors and read with `.iter_batches()` or `.iter_rows()`. This does not incur a copy, since the blocks of the Dataset are passed by reference as Ray objects. Splitting data as shards and passing to individual Ray Actors to process shards is a common Ray pattern used in distributed training with Ray actors.

Let's examine how we can process a list of shards with a `BatchWorker` Actor  in a distributed fashion

<img src="images/batch_worker.jpg" width="80%" height="35%">

A Ray actor `BatchWorker` working through shards in batche size of 1024.

In [36]:
@ray.remote
class BatchWorker:
    def __init__(self, rank):
        self.rank = rank         # this could be rank of CPU/GPU or worker id
        self.processed = 0       # how much was processed
    
    @ray.method(num_returns=2)   # we want to return a tuple
    def process_shard_list(self, shard: ray.data.Dataset) -> tuple:
        for batch in shard.iter_batches(batch_size=1024):
            # here you could do something with the batch such as feature
            # preprocessing, minor transformation, and then
            # save as a parquet file 
            self.processed = self.processed + len(batch)
        # return items processed, worker id
        return (self.processed, self.rank)     

#### Create batch workers as Ray actors
Each actor will get a shard, list of rows, to work on. We split
our dataset `arrow_ds` into five shards. Each `BatchWorker` gets a shard.

`.split`() splits shards across these batch of workers by using the `locality_hints`.

In [39]:
# create five actors as BatchWorker
batch_workers = [BatchWorker.remote(i) for i in range(1, 6)]

#split into five shards, each one for an actor
shards = arrow_ds.split(len(batch_workers), locality_hints=batch_workers)

print(f"Shard row: {shards[0]}")
print(f"Number of shards:{len(shards)}")
print(f"Number of shard workers:{len(batch_workers)}")

Shard row: Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string})
Number of shards:5
Number of shard workers:5


### Launch `BatchWorker` actors

Process each shard. Each `BatchWorker.process_shard_list()` returns a object RefID with a tuple as its value. What we get from this comprehension is a list objectRefs as tuples.

In [40]:
object_refs = [w.process_shard_list.remote(s) for w, s in zip(batch_workers, shards)]
object_refs, len(object_refs)

([[ObjectRef(a46bb8530f4064fd96fe3c7e0a827f0c7257c41c0100000001000000),
   ObjectRef(a46bb8530f4064fd96fe3c7e0a827f0c7257c41c0100000002000000)],
  [ObjectRef(c0ca37ae352c9211edae489be93b199f324a6ada0100000001000000),
   ObjectRef(c0ca37ae352c9211edae489be93b199f324a6ada0100000002000000)],
  [ObjectRef(798376f73f71e22c0133d30135ec25e606e307f00100000001000000),
   ObjectRef(798376f73f71e22c0133d30135ec25e606e307f00100000002000000)],
  [ObjectRef(d99fa8d5c923d7c84186ff2664a0e3a7a30d0ff30100000001000000),
   ObjectRef(d99fa8d5c923d7c84186ff2664a0e3a7a30d0ff30100000002000000)],
  [ObjectRef(aa5ec071964645d2ba808c0411294a7ecf6129730100000001000000),
   ObjectRef(aa5ec071964645d2ba808c0411294a7ecf6129730100000002000000)]],
 5)

Fetch the values from the returned list of ObjectRefs, which is a tuple of (batch_size, worker_rank).

In [41]:
values = [ray.get(ref) for ref in object_refs]
values

[[160000, 1], [160000, 2], [160000, 3], [160000, 4], [160000, 5]]

### Creating and using Ray dataset pipelines

What are dataset pipelines and how are they different from Ray datasets? 

Datasets perform transformation or operations eagerly or synchronously, whereas [DataPipelines](https://docs.ray.io/en/latest/data/package-ref.html#datasetpipeline-api) can execute in an overlapped pipeline executions. For example, if you had operations that require reading from file, transforming data, and then doing some minor feature engineering, these operations can be executed in a normal pipeline fashion. This allows for the overlapped execution of data input (e.g., reading files), computation (e.g. feature preprocessing), and training (e.g., distributed ML training). 

<img src="images/pipeline_window.jpg" width="70%" height="35%">

A `DatasetPipeline` can be constructed in two ways: either by pipelining the execution of an existing Dataset (via `Dataset.window`) or generating repeats of an existing Dataset (via `Dataset.repeat`). 

Let's have a go at it and see what we can do with our synthetic data from above.


### Using Dataset.window

Create some functions or operations to be executed in a overlapped manner in the pipeline. These functions
are simple to illustrate a point. But they can be complex for a particular use case.


In [42]:
def divide_row_value(row: ray.data.impl.arrow_block.ArrowRow, n) -> int:
    return round(row / n)

In [43]:
def double_row_value(row: ray.data.impl.arrow_block.ArrowRow, n) -> int:
    return row * n

In [44]:
def modulo_row_value(row: ray.data.impl.arrow_block.ArrowRow, n) -> int:
    return row % random.randint(1, n)

#### Create a window based pipeline
With a each window of 50 blocks. 

In [59]:
# Use our original simple dataset from above with 1K rows in integer
ds_pipe = ds.window(blocks_per_window=50)
ds_pipe

DatasetPipeline(num_windows=1, num_stages=1)

### Applying transforms to pipelines adds more pipeline stages.

In [60]:
ds_pipe = ds_pipe.map(lambda row: divide_row_value(row, 2))
ds_pipe = ds_pipe.map(lambda row: double_row_value(row, 3))
ds_pipe = ds_pipe.map(lambda row: modulo_row_value(row, 4))
print(ds_pipe)

DatasetPipeline(num_windows=1, num_stages=4)


#### Iterate our pipeline

 * _Questions for clearification_:
     * _how is this executed_?
     * _why are we iterating over rows_?
     * _what is row comprised of? Blocks?_?
     * _is the value of the row already computed value_?
     * _if the `num_stages=4`, why am I seeing only stage 0 and 1 in the output of stages?_

In [61]:
results=[]
for row in ds_pipe.iter_rows():
    results.append(row)
print(f"Results from each pipeline map function:{results}")
print(f"Total value: {sum(results)}")

Stage 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.28it/s]

Results from each pipeline map function:[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 2, 0]
Total value: 11


Let's try a `Datapipeline` with our synthetic data *Homewowners*

In [62]:
# count or return based on the condition
def count_state(row: ray.data.impl.arrow_block.ArrowRow, state) -> int:
    return 1 if row['state'] == state and row["defaulted"] else 0

In [64]:
arrow_ds_pipe = arrow_ds.window(blocks_per_window=50)
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=2)

In [65]:
arrow_ds_pipe = arrow_ds_pipe.map(lambda row: count_state(row, "CA"))
arrow_ds_pipe

DatasetPipeline(num_windows=1, num_stages=3)

In [66]:
results=[]
for row in arrow_ds_pipe.iter_rows():
    results.append(row)
print(f"Total rows for CA state and defaulted loans rows: {sum(results)}")

Stage 0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.92s/it]

Total rows for CA state and defaulted loans rows: 49733


## Ingesting data into Model Trainers
Let's define a toy `Trainer` actor that takes our synthetic data and trains the model and returns loss for that trainer. This is common pattern
in Ray for distributing data to Trainers in a Ray cluster.

<img src="images/trainer_worker.jpg" width="75%" height="40%">

In [73]:
# Our dummy model
def model(input):
    return random.uniform(0, 1)

@ray.remote
class Trainer:
    def __init__(self, rank, model):
        self.rank = rank
        self.model = model
        self.loss = 0.0
        
    def train(self, shard:ray.data.Dataset) -> float:
        for epoch in range(1,21):
            for batch in shard.iter_batches(batch_size=1024):
                output = self.model(batch)
                self.loss = output 
            if epoch % 5 == 0:
                print(f'rank: {self.rank} epoch: {epoch}, loss: {self.loss:.3f}')
        return self.loss

#### Create five trainers, each with a copy of the model and each training on its respective shard

In [74]:
trainers = [Trainer.remote(i, model) for i in range(1, 6)]
trainers

[Actor(Trainer, b789d03e6ce2e67f4adbdc7101000000),
 Actor(Trainer, 74e37eb767d463041f4bea6901000000),
 Actor(Trainer, e81ce61112061595e85aa19f01000000),
 Actor(Trainer, 4c6fc39f4679a5ed22c31a4d01000000),
 Actor(Trainer, e1fb906dcf7921765c09d36601000000)]

#### Split the shards across all trainers

In [75]:
shards = arrow_ds.split(n=len(trainers), locality_hints=trainers)
shards

[Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: null, name: null, amount: double, interest: double, state: string, marital_status: string, property: string, dependents: int64, defaulted: int64, gender: string}),
 Dataset(num_blocks=1, num_rows=160000, schema={id: int64, ssn: null, name: null, amount

#### Launch our trainers in a distributed fashion

This will run across the cluster. Check the dashbard to see five actors launched. On a cluster, they will be on five different nodes, whereas on a single node on  
five different cores.

In [76]:
object_refs = [t.train.remote(s) for t, s in zip(trainers, shards)]

(Trainer pid=56118) rank: 2 epoch: 5, loss: 0.397
(Trainer pid=56117) rank: 1 epoch: 5, loss: 0.876
(Trainer pid=56120) rank: 4 epoch: 5, loss: 0.211
(Trainer pid=56119) rank: 3 epoch: 5, loss: 0.612
(Trainer pid=56121) rank: 5 epoch: 5, loss: 0.803
(Trainer pid=56118) rank: 2 epoch: 10, loss: 0.651
(Trainer pid=56119) rank: 3 epoch: 10, loss: 0.244
(Trainer pid=56117) rank: 1 epoch: 10, loss: 0.560
(Trainer pid=56120) rank: 4 epoch: 10, loss: 0.132
(Trainer pid=56121) rank: 5 epoch: 10, loss: 0.251
(Trainer pid=56118) rank: 2 epoch: 15, loss: 0.039
(Trainer pid=56119) rank: 3 epoch: 15, loss: 0.856
(Trainer pid=56117) rank: 1 epoch: 15, loss: 0.805
(Trainer pid=56120) rank: 4 epoch: 15, loss: 0.261
(Trainer pid=56121) rank: 5 epoch: 15, loss: 0.798
(Trainer pid=56118) rank: 2 epoch: 20, loss: 0.696
(Trainer pid=56119) rank: 3 epoch: 20, loss: 0.999
(Trainer pid=56117) rank: 1 epoch: 20, loss: 0.055
(Trainer pid=56120) rank: 4 epoch: 20, loss: 0.220
(Trainer pid=56121) rank: 5 epoch: 2

In [77]:
ray.get(object_refs)

[0.05453356322433289,
 0.6960078710707507,
 0.9987707494051961,
 0.21978954355860314,
 0.8585152715233534]

In [78]:
ray.shutdown()

### Exercises
 1. Write some simple transformers, filters, and aggregators with our synthetic data. For example:
  * use [`.add_column()`](https://docs.ray.io/en/master/data/package-ref.html) to add an `age` column
  * filter by gender == 'U'
  * aggregate (or groupby `property`) and count each. 
 2. Add additional pipleline stages function `def count_tx(...)` with our synthetic data. For example, count all people in state of `TX`, `married` and `defaulted`.

### Homework

So far we have covered the basics of Ray Datasets. There are advanced topics that you can now explore since you know the basics. Below is a list of tasks you will want to work through at home.

1. Work through the [NYC example tutorial](extra/ray_data_nyc.ipynb). This explores how you use `.map_batches()` for filtering and map operations using vectorized UDFs
2. Peruse the user guides for advanced examples in [data transformation](https://docs.ray.io/en/master/data/transforming-datasets.html#transforming-datasets) and [ML preprocessing](https://docs.ray.io/en/master/data/dataset-ml-preprocessing.html#datasets-ml-preprocessing)
3. Read how to do large scale [ML ingest](https://docs.ray.io/en/master/data/examples/big_data_ingestion.html)
4. Advanced [pipeline usage](https://docs.ray.io/en/latest/data/advanced-pipelines.html#)